# 1. Information about the submission

## 1.1 Name and number of the assignment

***Enter here** name and number of the assignment*

## 1.2 Student name

***Enter here** your Name and Surname*

## 1.3 Codalab user ID

***Enter here** your Codalab user ID used to submit your run to shared task*

## 1.4 Additional comments

***Enter here** any additional comments which you would like to communicate to a TA who is going to grade this work not related to the content of your submission.*

# 2. Technical Report

*Use Section 2 to describe results of your experiments as you would do writing a paper about your results. DO NOT insert code in this part. Only insert plots and tables summarizing results as needed. Use formulas if needed do described your methodology. The code is provided in Section 3.*

## 2.1 Methodology

***Enter here** a detailed description of the method used in your submission(s) to Codalab. The description should be at least 2-4 paragraphs featuring the following: type of the model, meta-parameters, how did you select meta-parameters, any further modifications of the out-of-the-box solutions, etc. The text is markdown and you can use math environment to write formulas:*

$\hat{y}=\beta_0 + \sum^p_{j=1} x_j \beta_j$

Also you can insert images as needed:

![image](https://upload.wikimedia.org/wikipedia/commons/6/6d/Exam_pass_logistic_curve.jpeg)

*This part of the should contain description of all methods that you tried and, most importantly, that worked the best for you. Here you can include some tricks of your preprocessing, description of the models and motivation of their usage, the description of the training process details (train-test split, cross-validation, etc.). So, everything valuable that will help us to understand the scope of your work and reproduce your pipeline*

*The 'innovativeness' scores will be assigned based on the content of this part. In case you just use a 'drop-in' baseline model these scores will be low. In case you made some clever modification of the model which improves the result this score will be low. However, it does not make sense do describe some modifications which are creative but does not work at all. Try different approaches, models, play with preprocessing, hyperparameters, etc. It is OK to reimplement some already existing approach. It is OK that some of your experiments did not work as you expected. Show us that you used your creativity and ran several experiments.*

## 2.2 Discussion of results

***Enter here** a discussion of results and a summary of the experiment. Here we want to see the final table with comparison of the baseline and all tried approaches you decided to report. Even if some method did not bring you to the top of the leaderboard, you should nevertheless indicate this result and a discussion, why, in your opinion, some approach worked and another failed. Interesting findings in the discussion will be a plus.*

Method | Precision | Recall
--- | --- | ---
Baseline | 0.88 | 0.77
My great method 1 | 0.99 | 0.11
My great method 2 | 0.90 | 0.90

*If relevant insert plots and historgams in this section e.g. testing variation of the score with respect to some parameters e.g. learning rate or size of the input dataset, etc. Please do not use code to generate plots, instead just insert images as shown below. Plots could be generated from code in Section 3. *

![image](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Sine_and_Cosine.svg/640px-Sine_and_Cosine.svg.png)

# 3. Code

*Enter here all code used to produce your results submitted to Codalab. Add some comments and subsections to navigate though your solution.*

*In this part you are expected to develop yourself a solution of the task and provide a reproducible code:*
- *Using Python 3;*
- *Contains code for installation of all dependencies;*
- *Contains code for downloading of all the datasets used*;
- *Contains the code for reproducing your results (in other words, if a tester downloads your notebook she should be able to run cell-by-cell the code and obtain your experimental results as described in the methodology section)*.


*As a result, you code will be graded according to these criteria:*
- ***Readability**: your code should be well-structured preferably with indicated parts of your approach (Preprocessing, Model training, Evaluation, etc.).*
- ***Reproducibility**: your code should be reproduced without any mistakes with “Run all” mode (obtaining experimental part).*


## 3.1 Requirements

In [ ]:
!pip install spacy
!pip install torchmetrics
!pip install --upgrade torch torchtext
!pip install transformers
!pip install torchmetrics
!python3 -m spacy download ru_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-21 12:21:32.116780: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 12:21:33.462386: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.de

In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn

from tqdm.notebook import tqdm
import os
import pickle
from collections import Counter, OrderedDict
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


from torchtext.vocab import Vocab
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import torchmetrics
from torchmetrics.classification import BinaryF1Score
from transformers import AutoTokenizer, AutoModel

import spacy

## 3.2 Download the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if 'data' not in os.listdir():
  os.mkdir('data')

In [ ]:
os.chdir('data')

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!wget -O train.tsv https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/train.tsv
!wget -O val_empty.tsv https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/val_empty.tsv
!wget -O test-no_labels.tsv https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/test-no_labels.tsv

--2023-04-21 12:22:01--  https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1539551 (1.5M) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>]   1.47M  --.-KB/s    in 0.04s   

2023-04-21 12:22:01 (39.2 MB/s) - ‘train.tsv’ saved [1539551/1539551]

--2023-04-21 12:22:02--  https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/val_empty.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308303 (301K) [text/pl

In [ ]:
train_df = pd.read_csv('train.tsv', sep='\t')
train_df, val_df = train_test_split(train_df, random_state=42)
train_df.head()

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
2142,22503,"При этом маски никто не носит, руки не дезинфи...",1,1,-1,-1,-1,-1
4367,28102,"Вы у медиков - хирургов спросите, как они по 1...",1,1,-1,-1,-1,-1
564,18525,"Ну, так никто не предполагал, что толпы обезум...",-1,-1,1,1,-1,-1
478,18295,Кто будет обеспечивать людей которые сидят на ...,-1,-1,0,0,-1,-1
2229,22705,"Она налицо: все люди в масках, график заболева...",1,0,-1,-1,-1,-1


## 3.3 Preprocessing

In [ ]:
target_col = ['masks_stance',
              'masks_argument',
              'quarantine_stance',
              'quarantine_argument',
              'vaccines_stance',
              'vaccines_argument']
train_df[target_col].apply(pd.Series.value_counts)

,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
-1,2688,2688,3474,3474,3774,3774
0,445,239,130,88,309,197
1,1369,1856,1014,1314,661,953
2,535,254,419,161,293,113


In [ ]:
STANCE_LABELS = [2,1,0,-1]
ARG_LABELS = [2,1,0,-1]
ALL_LABELS = ARG_LABELS + STANCE_LABELS

LABEL_DICTIONARY = {
    2: 'for',
    1: 'other',
    0: 'against',
    -1: 'irrelevant'
}

id2label = {k:l for k, l in enumerate(ALL_LABELS)}
label2id = {l:k for k, l in enumerate(ALL_LABELS)}

In [ ]:
def preprocess_function(df,opt='masks'):
  labels = [0]*len(id2label)
  target_col = [f'{opt}_stance',f'{opt}_argument']
  check = len(list(id2label.items()))
  for k,l in id2label.items():
    if (l == df[target_col[0]]) and (k<check//2):
      labels[k] = 1
    elif (l == df[target_col[1]]) and (k>=check//2):
      labels[k] = 1
    else:
      labels[k] = 0
  df['labels'] = labels
  return df

In [ ]:
train_databases = dict()
val_databases = dict()
cases = ['masks','quarantine','vaccines']
options = ['argument','stance']
for key in cases:
  columns = ['text',f'{key}_stance',f'{key}_argument',]

  train = train_df[columns]
  train['labels'] = pd.NA
  train = train.apply(preprocess_function,args=(key,),axis=1)
  train = train.drop([f'{key}_stance',f'{key}_argument'],axis=1)
  train_databases[key] = train

  val = val_df[columns]
  val['labels'] = pd.NA
  val = val.apply(preprocess_function,args=(key,),axis=1)
  val = val.drop([f'{key}_stance',f'{key}_argument'],axis=1)
  val_databases[key] = val


<ipython-input-41-694c09d73654>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['labels'] = pd.NA
<ipython-input-41-694c09d73654>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['labels'] = pd.NA
<ipython-input-41-694c09d73654>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [ ]:
train_databases['masks'].head()

,text,labels
2142,"При этом маски никто не носит, руки не дезинфи...","[0, 1, 0, 0, 0, 1, 0, 0]"
4367,"Вы у медиков - хирургов спросите, как они по 1...","[0, 1, 0, 0, 0, 1, 0, 0]"
564,"Ну, так никто не предполагал, что толпы обезум...","[0, 0, 0, 1, 0, 0, 0, 1]"
478,Кто будет обеспечивать людей которые сидят на ...,"[0, 0, 0, 1, 0, 0, 0, 1]"
2229,"Она налицо: все люди в масках, график заболева...","[0, 1, 0, 0, 0, 0, 1, 0]"


### ML model

In [ ]:
class CommentsDataset(Dataset):
  def __init__(self,data, tokenizer, max_token_len,targets):
    self.tokenizer = tokenizer
    self.data = data
    self.max_token_len = max_token_len
    self.targets = targets

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    data_row = self.data.iloc[index]
    comment_text = data_row.text
    if self.targets != None:
      labels = data_row[self.targets]
    else:
      labels = 0
    encoding = self.tokenizer(
      comment_text,
      add_special_tokens=True,
      max_length=self.max_token_len,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return dict(
      comment_text=comment_text,
      input_ids=encoding["input_ids"].flatten(),
      attention_mask=encoding["attention_mask"].flatten(),
      labels=torch.FloatTensor(labels)
    )

In [ ]:
class CommentTagger(nn.Module):

  def __init__(self, n_classes):
    super().__init__()
    self.bert = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    output = self.bert(input_ids, attention_mask=attention_mask)
    output = self.classifier(output.pooler_output)
    output = torch.sigmoid(output)
    return output

In [ ]:
def run_epoch(DataLoader,model,optimizer,scheduler,mode,device,
              criterion=torch.nn.functional.binary_cross_entropy_with_logits,
              metric=BinaryF1Score()):

  if mode == 'train':
    model.train()
    total_train_loss = 0
    for i, batch in tqdm(enumerate(DataLoader)):
      inp_ids = batch["input_ids"].to(device)
      att_mask = batch["attention_mask"].to(device)
      label = batch["labels"].to(device)
      out = model(inp_ids,att_mask)
      loss = criterion(out,label)
      loss.backward()
      optimizer.step()
      if scheduler!=None:
        scheduler.step(optimizer)
      optimizer.zero_grad()
      total_train_loss += loss.item()
    return total_train_loss/len(DataLoader),0

  elif mode == 'val':
    model.eval()
    metric.to(device)
    total_val_loss = 0
    total_f1_score= 0
    for i, batch in tqdm(enumerate(DataLoader)):
      inp_ids = batch["input_ids"].to(device)
      att_mask = batch["attention_mask"].to(device)
      label = batch["labels"].to(device)
      out = model(inp_ids,att_mask)
      loss = criterion(out,label)
      preds = torch.where(out>0.5,1,0)
      f1 = metric(preds,label)
      total_val_loss += loss.item()
      total_f1_score += f1
    return total_val_loss/len(DataLoader),total_f1_score/len(DataLoader)




In [ ]:
def train(n_epochs,DataLoaders,model,optimizer,scheduler,device,path=None):
  modes = ['train','val']
  f1_history = []

  for epoch in range(1,n_epochs+1):
    print('\n' + '-'*40)
    print(f'Epoch: {epoch}')
    print(f'Run training...')
    for mode in modes:
      train_loss,f1_score = run_epoch(DataLoaders[mode],model,optimizer,scheduler,mode,device)
      if mode == 'train':
        print('End of epoch')
        print('Accumulated train loss: ',train_loss)
      elif mode == 'val':
        f1_history.append(f1_score)
        print('Accumulated val loss: ',train_loss)
        print('Accumulated val f1_score: ',f1_score)
    #if epoch%2==0:
      #save_checkpoint(epoch, model, path)
  return f1_history



In [ ]:
batch_size = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

models = dict()
os.chdir('..')

for key in train_databases.keys():
  print(key)
  target_col = train_databases[key].columns.to_list()[2:]

  train_dataset = CommentsDataset(train_databases[key],tokenizer,max_token_len=200,targets = target_col)
  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  val_dataset = CommentsDataset(val_databases[key],tokenizer,max_token_len=200,targets = target_col)
  val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

  dataloaders = {"train": train_dataloader, "val": val_dataloader}

  model = CommentTagger(n_classes=len(target_col),label_columns=target_col).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
  #scheduler = torch.optim.lr_scheduler.StepLR(optimizer = optimizer, step_size=4, gamma=0.1)
  scheduler = None

  for param in model.bert.parameters():
    param.requires_grad = False

  n_epochs=5
  hist = train(n_epochs,dataloaders,model,optimizer,scheduler,device)

  torch.save(model.state_dict(),f'drive/MyDrive/{key}model')


quarantine_stance



----------------------------------------
Epoch: 1
Run training...


1008it [03:44,  4.50it/s]


End of epoch
Accumulated train loss:  0.4806135678516021


336it [01:14,  4.51it/s]


...

Accumulated val loss:  0.2836430321358854
Accumulated val f1_score:  tensor(0.7839, device='cuda:0')

----------------------------------------
Epoch: 5
Run training...


1008it [03:46,  4.44it/s]


End of epoch
Accumulated train loss:  0.29329429334029555


336it [01:14,  4.50it/s]


Accumulated val loss:  0.2718296128635605
Accumulated val f1_score:  tensor(0.7916, device='cuda:0')


In [ ]:
model_name = 'masks_stancemodel'

In [ ]:
os.chdir('..')

In [ ]:
model_param = torch.load(f'drive/MyDrive/{model_name}')

In [ ]:
#target_col = train_databases['masks_stance'].columns.to_list()[2:]
device = torch.device('cuda')
model = CommentTagger(n_classes=4,label_columns=None).to(device)

In [ ]:
model.load_state_dict(model_param)

<All keys matched successfully>

In [ ]:
test_df = pd.read_csv('data/test-no_labels.tsv', sep='\t')

In [ ]:
test_df = test_df[['text']]

In [ ]:
batch_size = 5
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
test_dataset = CommentsDataset(val_df,tokenizer,max_token_len=200,targets = None)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
val_databases['masks_stance'].head(10)

,text_id,text,masks_stance_-1,masks_stance_0,masks_stance_1,masks_stance_2
5857,31889,"Да и хрен то с этим , живы после вакцинации , ...",1,0,0,0
4220,27739,Только раньше маски должны были надевать больн...,0,0,0,1
2484,23343,"[USER], увлажняйте глаз, если появился синдром...",0,0,0,1
1615,21190,"В торговые центры на карантине пойдут те, кто ...",1,0,0,0
4149,27563,А маски реально помогают если их правильно нос...,0,0,0,1
3328,25471,"[USER], парадокс ситуации в том что я без маск...",0,1,0,0
743,18977,"[USER], я же сказала , что не правильно вырази...",1,0,0,0
5547,31110,"Уж извольте, я лучше сиз носить буду в транспо...",1,0,0,0
5527,31064,"По словам Гейтса, он надеется, что работающая ...",1,0,0,0
4081,27393,"У одного нос торчит из-под маски, другой рукам...",0,1,0,0


In [ ]:
predictions

tensor([[[1, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 0, 0, 0]],

        ...,

        [[1, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0]]])

In [ ]:
predictions = []
for i, batch in tqdm(enumerate(test_dataloader)):
    inp_ids = batch["input_ids"].to(device),
    att_mask = batch["attention_mask"].to(device)
    out = model(inp_ids[0],att_mask)
    #preds = torch.where(out>0.5,1,0).cpu()
    predictions.append(out.cpu())

predictions = torch.stack(predictions)
#print(predictions.flatten().reshape(15,4))

5it [00:01,  3.69it/s]


OutOfMemoryError: ignored

In [ ]:
n_epochs=10
path = 'chlp'
hist = train(n_epochs,dataloaders,model,optimizer,scheduler,path,device)


----------------------------------------
Epoch: 1
Run training...


1008it [03:41,  4.54it/s]


End of epoch
Accumulated train loss:  0.5295598658895682


336it [01:12,  4.63it/s]


Accumulated val loss:  0.4504329395436105
Accumulated val f1_score:  tensor(0.6084, device='cuda:0')

----------------------------------------
Epoch: 2
Run training...


...

Accumulated val loss:  0.328284324900735
Accumulated val f1_score:  tensor(0.6907, device='cuda:0')

----------------------------------------
Epoch: 10
Run training...


1008it [03:41,  4.56it/s]


End of epoch
Accumulated train loss:  0.33029656055279905


336it [01:12,  4.65it/s]

Accumulated val loss:  0.3222084776720121
Accumulated val f1_score:  tensor(0.6957, device='cuda:0')
